In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

batch_size = 256
image_size = 96

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = STL10(root='./data', split='train', transform=transform, download=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

nc = 3
ndf = 96
num_epochs = 100
lr = 0.001
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)

class Encoder(nn.Module):
    def __init__(self, nc, ndf, dim_z, num_classes, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, dim_z, 6, 1, 0, bias=False)  
        )
        self.fc = nn.Linear(dim_z, num_classes)  

    def forward(self, input):
        latent = self.main(input)
        latent = latent.view(input.size(0), -1) 
        x = self.fc(latent)
        return x, latent


model = Encoder(ngpu=n_gpu, nc=nc, ndf=ndf, dim_z=2, num_classes=10)
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0)

for epoch in range(num_epochs):
    nlpca_features = []  
    for i, (data, labels) in enumerate(train_loader, 0):
        data_real = data.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        x, latent = model(data_real)
        nlpca_features.append(latent.detach().cpu().numpy())

        loss = criterion(x, labels)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    nlpca_features = np.concatenate(nlpca_features, axis=0)
    np.save(f'nlpca_features_epoch_{epoch + 1}.npy', nlpca_features)
    plt.figure(figsize=(8, 8))
    plt.scatter(nlpca_features[:, 0], nlpca_features[:, 1], c=labels.cpu().numpy(), cmap='viridis')
    plt.title(f'2D Nonlinear PCA Features - Epoch {epoch + 1}')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.colorbar()
    plt.savefig(f'nlpca_features_scatter_epoch_{epoch + 1}.png')
    plt.close()

    torch.save(model.state_dict(), f'encoder_model_epoch_{epoch + 1}.pth')

cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
